# 본 파일은 데이터 전처리 결과 비교를 위한 RAG 파이프라인

- 5/25 회의

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from dotenv import load_dotenv

load_dotenv()

# 파일 읽기
original_file_path = "/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/self/Apm005-03-18.md"
edit_file_path = "/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/self/Apm005-03-18(전처리).md"

with open(original_file_path, "r", encoding="utf-8") as f:
    origin = f.read()

with open(edit_file_path, "r", encoding="utf-8") as f:
    edit = f.read()

# 청크 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "."])

origin_texts = text_splitter.create_documents([origin])
edit_texts = text_splitter.create_documents([edit])

# 크로마 저장
origin_path = "./origin_db"
edit_path = "./edit_db"

# save_origin_db = Chroma.from_documents(
#     origin_texts, 
#     OpenAIEmbeddings(model="text-embedding-3-large"), 
#     persist_directory=origin_path , 
#     collection_name="origin")

# save_edit_db = Chroma.from_documents(
#     edit_texts, 
#     OpenAIEmbeddings(model="text-embedding-3-large"), 
#     persist_directory=edit_path, 
#     collection_name="edit")

load_origin_db = Chroma(
    persist_directory=origin_path,
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"),
    collection_name="origin")

load_edit_db = Chroma(
    persist_directory=edit_path,
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"),
    collection_name="edit")

# 리트리버
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor_retriever = CrossEncoderReranker(model=reranker_model, 
                                            top_n=5)

origin_db_retriever = load_origin_db.as_retriever(search_kwargs={"k": 10})
origin_retriever = ContextualCompressionRetriever(base_retriever=origin_db_retriever, 
                                                    base_compressor=compressor_retriever)

edit_db_retriever = load_edit_db.as_retriever(search_kwargs={"k": 10})
edit_retriever = ContextualCompressionRetriever(base_retriever=edit_db_retriever, 
                                                    base_compressor=compressor_retriever)

In [3]:
from typing import Annotated, List, TypedDict
from langgraph.graph.message import add_messages
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline

# GraphState 상태 정의
class ChatbotState(TypedDict):
    question: Annotated[str, "Question"]  # 질문
    documents: Annotated[List, "Context"]  # 문서의 검색 결과
    chatbot: Annotated[str, "Answer"]  # 답변
    messages: Annotated[List, add_messages]  # 메시지(누적되는 list)

model = ChatOpenAI(
    temperature=0.3,
    model_name="gpt-4o-mini")

In [4]:
gpt_prompt = PromptTemplate.from_template("""
당신은 소아마취 전문 의료인입니다. 다음의 문서는 실제 환자에 대한 마취 증례 보고서입니다. 

질문에 대해 아래 문서에서 관련 있는 정보만을 근거로 하여 명확하고 간결하게 답변하세요.  
답변은 전문적인 의학 용어와 신중한 어조로 작성되어야 하며, 문서에서 직접적으로 언급되지 않은 정보에 대해서는 “문서에 해당 내용은 명시되어 있지 않습니다”라고 답변하세요.  
질문이 매우 구체적이거나 수치 기반일 경우, 문서 속 수치와 진술을 근거로 사용하세요.  
문서와 무관하거나 추론이 불가능한 질문은 답변하지 마세요.

### 질문:
{user_question}

### 참고 문서:
{retrieved_context}

### 답변:
""")

In [10]:
from langchain_community.document_transformers import LongContextReorder
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END

memory = MemorySaver()
reorder = LongContextReorder()

# 문서 검색 노드
def retrieve_document(state: ChatbotState):
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 문서에서 검색하여 관련성 있는 문서를 찾습니다.
    retrieved_docs = edit_retriever.invoke(latest_question)

    ordered_context = reorder.transform_documents(retrieved_docs)

    # 검색된 문서를 context 키에 저장합니다.
    return ChatbotState(documents=ordered_context)


# 답변 생성 노드
def llm_answer(state: ChatbotState):
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 검색된 문서를 상태에서 가져옵니다.
    docs = state["documents"]

    formatted_prompt = gpt_prompt.format(user_question=latest_question, retrieved_context=docs)

    # LLM 체인을 호출하여 답변 생성 (llm_chain은 사용 환경에 맞게 초기화되어야 합니다.)
    response = model.invoke(formatted_prompt)   

    # model.invoke가 이미 AIMessage를 반환한다면, 이를 문자열로 변환합니다.
    if isinstance(response, AIMessage):
        response_text = response.content
    else:
        response_text = str(response)
    
    # 생성된 답변, (유저의 질문, 답변) 메시지를 상태에 저장합니다.
    return ChatbotState(chatbot=response, 
                    messages=[("user", latest_question), ("assistant", response_text)])

graph_builder = StateGraph(ChatbotState)

graph_builder.add_node("docs", retrieve_document)
graph_builder.add_node("llm_answer", llm_answer)

graph_builder.add_edge(START, "docs")
graph_builder.add_edge("docs", "llm_answer")
graph_builder.add_edge("llm_answer",END)

graph = graph_builder.compile(checkpointer=memory)

In [ ]:
# origin(전처리 안한)
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id" : "1"})

question = "수술 중 칼슘 수치가 위험한 수준으로 떨어진 시점은 언제인가요?"

for event in graph.stream(ChatbotState(question=question), config=config):
    for value in event.values():
        if 'documents' in value:
            for idx, doc in enumerate(value['documents']):  # 리스트 순회
                print(f"📄 {idx+1}번째 문서")
                print(doc.page_content)

        if 'chatbot' in value:
            print("💬 답변:")
            print(value['chatbot'].content)

📄 1번째 문서
. 수술 중 Na\n는 135- 138 mM, K+는 3.5-3.7 mM 로 유지되었고, 수술\n끝나기 전 마지막 검사소견 상 Ca2+는 대량수혈로 인해\n1.18 mg/dl에서 0.87 mg/dl로 감소된 소견을 보여 10% calcium\ngluconate 5 ml를 정주하였고, pH 7.43, pCO2 38 mmHg, pO2\n265 mmHg, HCO3 27-28 mM, Hb 7
📄 2번째 문서
. 대량\n수혈로 발생 가능한 고칼륨혈증, 저칼슘혈증, 산증, 저체온\n증 등을 수술 중 조절하였다(Table 1) 수술 전 심박수 130회\n/분, 혈압 85/50 mmHg였고, 술 중 활력징후는 잘 유지되었\n기 때문에 승압제는 사용하지 않았다.\n\nTable 1
📄 3번째 문서
.\n\n3시간의 마취유지 중 환자의 총 혈액량의 3배인 1,500 ml\n이상 실혈이 발생하였고, 수술실에서 응급 동맥혈검사(i-STAT,\nAbbott, IL, USA)를 시행하여 Hb, Hct를 확인하고 중심정맥\n압과 suction bottle과 gauze의 출혈량을 확인하면서 수혈하\n였다. 수술 중 중심정맥압은 6-8 mmHg, Hb 6
📄 4번째 문서
. 영아인 환아의 요도손상을 방지하기 위해\n집뇨기(urine collector)를 부착하였고, 3시간의 수술 동안 100\nml의 소변이 측정되었으며, 수술 후 중환자실에서 30 ml/h\n로 측정되었다. 호기 말 이산화탄소와 동맥혈가스분석을 통\n해 산소, 이산화탄소, pH, 전해질(Na , K + Ca2+)을 정상치\n,\n로 유지하였다. 수술 끝나기 전 마지막 검사소견 상 Ca2+는\n대량수혈로 인해 1.18 mg/dl에서 0
📄 5번째 문서
.18 mg/dl에서 0.87 mg/dl로 감소된 소견\n을 보여 10% calcium gluconate 5 ml를 정주하였고, 체온은\n가온모포와 수액과 혈액의 가온을 통해 체온조절을 하였으\n나 37°C에서 36°C까지 감소했다. 대량수혈로 발생 가능한\n고

In [ ]:
# edit(전처리 한)

for event in graph.stream(ChatbotState(question=question), config=config):
    for value in event.values():
        if 'documents' in value:
            for idx, doc in enumerate(value['documents']):  # 리스트 순회
                print(f"📄 {idx+1}번째 문서")
                print(doc.page_content)

        if 'chatbot' in value:
            print("💬 답변:")
            print(value['chatbot'].content)

📄 1번째 문서
.7로 회복되는 양상을 보였다. 칼슘(Ca) 수치는 수술 전 9.1 mg/dL에서 시작해 1시간 및 2시간 시점에는 1.18로 급격히 저하되었으며, 2시간 30분 시점에는 0.87로 더욱 감소하였다.ABGA(동맥혈 가스 분석) 수치 변화는 다음과 같다. pH는 수술 전 7.47에서 1시간 후 7.38로 감소하였으나, 이후 2시간 시점에 7.46로 회복되었고 2시간 30분 시점에는 7.43을 기록하였다. **이산화탄소 분압(pCO₂)**는 수술 전 44 mmHg에서 시작하여 2시간 이후에는 38 mmHg로 감소하였다
📄 2번째 문서
. 호기 말 이산화탄소와 동맥혈가스분석을 통해 산소, 이산화탄소, pH, 전해질(Na , K + Ca2+)을 정상치로 유지하였다. 수술 끝나기 전 마지막 검사소견 상 Ca2+는 대량수혈로 인해 1.18 mg/dl에서 0.87 mg/dl로 감소된 소견을 보여 10% calcium gluconate 5 ml를 정주하였고, 체온은 가온모포와 수액과 혈액의 가온을 통해 체온조절을 하였으나 37°C에서 36°C까지 감소했다. 대량수혈로 발생 가능한 고칼륨혈증, 저칼슘혈증, 산증, 저체온증 등을 수술 중 조절하였다
📄 3번째 문서
. 수술 전 혈소판 수치는 7 ×10⁹/L로 매우 심한 감소 상태였다. **수술 후 1일차(POD 1)**에는 54로 증가하였고, **2일차(POD 2)**에는 110으로 상승하였다. **5일차(POD 5)**에는 165로 꾸준히 증가하였으며, **13일차(POD 13)**에는 204까지 도달하였다. 이후 경과 관찰 결과, 수술 2개월 시점에서는 혈소판 수치가 251로 최고치를 기록하였고, 3개월 시점에는 190으로 소폭 감소하였다.]
📄 4번째 문서
. 수술 중 중심정맥압은 6-8 mmHg, Hb 6.8 g/dl, Hct 21%로 유지하기 위해 하트만용액 700 ml, 20% 알부민 100ml, 400ml 농축적혈구 3 unit, 혈소판 5 unit, 신선냉동혈장 2 unit 투여하였고, 혈소판수치는

In [ ]:
from langchain_core.runnables import RunnableConfig


# Llama
config = RunnableConfig(configurable={"thread_id" : "1"})   # 스레드 ID 설정

question = "직접 경구용 항응고제(DOACs)를 복용 중인 환자에게 경막외 또는 척추마취를 시행할 때 출혈 위험을 줄이기 위한 적절한 중단 시점은 언제인가요?"

for event in graph.stream(ChatbotState(question=question),config=config):
    for value in event.values():
        if 'documents' in value:
            for idx, doc in enumerate(value['documents']):  # 리스트 순회
                print(f"📄 {idx+1}번째 문서 / 문서 이름 : {doc.metadata['document_name']} / 페이지 : {doc.metadata['page']}")
                print(doc.page_content)

        if "chatbot" in value:
            response = value["chatbot"]
            full_text = getattr(response, "content", response)  # AIMessage or str

            if isinstance(full_text, str) and "Answer:" in full_text:
                answer_only = full_text.split("Answer:", 1)[-1].strip()
                print("💬 추출된 답변:")
                print(answer_only)
            else:
                print("💬 전체 응답:")
                print(full_text)

# 6) Documents 구성

- metadata : document_name, page, summary
- content : markdown

- **analysis_results 파일 바로 쓰기**

In [ ]:
import os
import json
import glob

folder_path = 'C:/Users/user/Desktop/BOAZ_ADV/Wang_Gyu/GUIDELINES/analysis_results'

json_files = glob.glob(os.path.join(folder_path,'*.json'))

json_data = {}
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path,filename)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data[filename] = json.load(file)

print(json_data.keys())

dict_keys(['1-s2.0-S0952818016300204-main.json', '1-s2.0-S0952818022000101-main.json', '1-s2.0-S0952818022003701-main.json', '1-s2.0-S0952818024000333-main.json', '1-s2.0-S0952818024001375-main.json', '1-s2.0-S2352556824000626-main.json'])


In [ ]:
from langchain_core.documents import Document

# 페이지로 합치기
documents = []

for file_name,data in json_data.items():
    elements = data['elements']

    page_content = {}

    for elem in elements:

        if elem['category'] in ('footer','footnote','header'):
            continue

        page = elem['page']
        content = elem['content']['markdown']

        if page in page_content:
            page_content[page] += "\n" + content
        else:
            page_content[page] = content

    for page,combined_content in page_content.items():
        metadata = {'document_name' : file_name,
                    'page' : page}
        documents.append(Document(metadata=metadata, page_content=combined_content))

documents

[Document(metadata={'document_name': '1-s2.0-S0952818016300204-main.json', 'page': 1}, page_content='![image](/image/placeholder)\n\n# Review\nUse of direct oral anticoagulants with regional\nanesthesia in orthopedic patients☆\nGianluca Cappelleri MD a,⁎, Andrea Fanelli MD b\naAnaesthesia and Intensive Care Unit, Azienda Ospedaliera Istituto Ortopedico Gaetano Pini, 20122, Milan, Italy\nbAnaesthesia and Intensive Care Unit, Policlinico S. Orsola-Malpighi, 40138, Bologna, Italy\nReceived 28 May 2015; revised 5 January 2016; accepted 22 February 2016\nKeywords:\nDirect oral anticoagulant;\nMajor orthopedic surgery;\nNeuraxial anesthesia;\nPerioperative management;\nPostoperative management\nAbstract The use of direct oral anticoagulants including apixaban, rivaroxaban, and dabigatran, which are\napproved for several therapeutic indications, can simplify perioperative and postoperative management\nof anticoagulation. Utilization of regional neuraxial anesthesia in patients receiving antic

In [ ]:
# 페이지별 summary를 생성해서 metadata에 넣어주자
from langchain_openai import ChatOpenAI
import openai

api_key = os.environ['OPENAI_API_KEY']
openai.api_key = api_key

system_prompt = """You are an expert who reads documents thoroughly and provides accurate summaries. 
The document you will read is a **medical research paper**. 
If you find that the document has missing parts at the beginning or the end, focus on summarizing the middle content. 
Since the summary you generate will later be used for retrieval to answer user queries, do not add any personal insights beyond the document's content—simply summarize it as it is.
Also, provide five hypothetical questions based on the image that users can ask.
"""

# 생성된 결과를 담을 리스트를 초기화합니다.
generated_documents = []

for doc in documents:
    context = doc.page_content
    doc_name = doc.metadata['document_name']
    doc_page = doc.metadata['page']

    user_prompt_template = f"""This is the document you will summarize.: {context}
###
Output Format:

<summary>
[summary]
</summary>
<hypothetical_questions>
[hypothetical_questions]
</hypothetical_questions>

Output must be written in English.
"""
    completion = openai.chat.completions.create(
    model="gpt-4o", 
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_template}])
    
    # 생성된 결과 텍스트 추출 후 리스트에 추가합니다.
    generated_text = completion.choices[0].message.content.strip()
    print(f"{doc_name}/{doc} 생성 완료")
    generated_documents.append(generated_text)


In [ ]:
generated_documents

['<summary>\nThe research paper examines the use of direct oral anticoagulants (DOACs), like apixaban, rivaroxaban, and dabigatran, during regional anesthesia in patients undergoing major orthopedic surgeries. These anticoagulants are authorized for various therapeutic purposes and help simplify the perioperative and postoperative anticoagulation management. However, their use with regional neuraxial anesthesia has a potential, albeit low, risk of hematoma, which requires careful assessment. The study emphasizes the importance of understanding current clinical data on hemorrhagic risks associated with DOACs, providing anesthesiologists with guideline recommendations and best practices for managing these medications. Specific attention is given to patients with high thromboembolic risk or renal impairment, who require special consideration during anticoagulation management to balance thromboprophylaxis and bleeding risks.\n</summary>\n<hypothetical_questions>\n1. What are the risks asso

In [ ]:
# summary / hypothetical_questions metadata에 넣기
import re

final_docs = []

for idx,summary in enumerate(generated_documents):
    page_content = documents[idx].page_content
    metadata = documents[idx].metadata

    summary_match = re.search(r"<summary>(.*?)</summary>", summary, re.DOTALL)
    if summary_match:
        summary_text = summary_match.group(1).strip()
        metadata ['summary'] = summary_text

    questions_match = re.search(r"<hypothetical_questions>(.*?)</hypothetical_questions>", summary, re.DOTALL)
    if questions_match:
        questions_text = questions_match.group(1).strip()
        metadata ['hypothetical_questions'] = questions_text

    final_docs.append(Document(metadata=metadata, page_content=page_content))

In [ ]:
final_docs[0]

Document(metadata={'document_name': '1-s2.0-S0952818016300204-main.json', 'page': 1, 'summary': 'The research paper examines the use of direct oral anticoagulants (DOACs), like apixaban, rivaroxaban, and dabigatran, during regional anesthesia in patients undergoing major orthopedic surgeries. These anticoagulants are authorized for various therapeutic purposes and help simplify the perioperative and postoperative anticoagulation management. However, their use with regional neuraxial anesthesia has a potential, albeit low, risk of hematoma, which requires careful assessment. The study emphasizes the importance of understanding current clinical data on hemorrhagic risks associated with DOACs, providing anesthesiologists with guideline recommendations and best practices for managing these medications. Specific attention is given to patients with high thromboembolic risk or renal impairment, who require special consideration during anticoagulation management to balance thromboprophylaxis a

In [ ]:
# 페이지별 자르기
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=250,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=50,
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False)

# 각 Document 객체의 page_content와 metadata를 리스트로 분리합니다.
texts = [doc.page_content for doc in final_docs]
metadatas = [doc.metadata for doc in final_docs]

# 이제 텍스트 리스트와 metadata 리스트를 함께 전달합니다.
split_docs = text_splitter.create_documents(texts, metadatas)

In [ ]:
split_docs

[Document(metadata={'document_name': '1-s2.0-S0952818016300204-main.json', 'page': 1, 'summary': 'The research paper examines the use of direct oral anticoagulants (DOACs), like apixaban, rivaroxaban, and dabigatran, during regional anesthesia in patients undergoing major orthopedic surgeries. These anticoagulants are authorized for various therapeutic purposes and help simplify the perioperative and postoperative anticoagulation management. However, their use with regional neuraxial anesthesia has a potential, albeit low, risk of hematoma, which requires careful assessment. The study emphasizes the importance of understanding current clinical data on hemorrhagic risks associated with DOACs, providing anesthesiologists with guideline recommendations and best practices for managing these medications. Specific attention is given to patients with high thromboembolic risk or renal impairment, who require special consideration during anticoagulation management to balance thromboprophylaxis 

- 파인콘 DB에 적재

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
import time
from tqdm import tqdm
from uuid import uuid4
from langchain_pinecone import PineconeVectorStore

pinecone_api = ''

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=pinecone_api)

index_name = "boaz-adv"

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  # OpenAI의 임베딩 모델 사용
    api_key=api_key)

# 임베딩 모델 차원 확인 (예: OpenAI의 'text-embedding-ada-002'는 1536차원)
EMBEDDING_DIMENSION = 1536  # OpenAI 임베딩 모델 차원

# 인덱스가 이미 존재하는 경우 생성하지 않고 사용
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=EMBEDDING_DIMENSION,  # 임베딩의 차원을 설정 (OpenAI 임베딩의 차원)
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    # 인덱스가 준비될 때까지 대기
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# 인덱스 가져오기
index = pc.Index(index_name)

# PineconeVectorStore를 사용하여 벡터 스토어 생성
vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="page_content")

# 문서에 UUID 할당
uuids = [str(uuid4()) for _ in range(len(split_docs))]

# 벡터 스토어에 문서 추가 (진행 상황 표시)
for doc, uuid in tqdm(zip(split_docs, uuids), total=len(split_docs), desc="Pinecone에 문서 추가 중"):
    vectorstore.add_documents(documents=[doc], ids=[uuid])

print("문서 저장이 완료되었습니다.")

Pinecone에 문서 추가 중: 100%|██████████| 1781/1781 [25:51<00:00,  1.15it/s] 

문서 저장이 완료되었습니다.


In [ ]:
vectorstore.asimilarity_search("What is the review about direct oral anticoagulants?")

<coroutine object VectorStore.asimilarity_search at 0x000001A1CCDDB6A0>

In [ ]:
# 적재된 Pinecone DB에서 데이터 불러오기
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# langsmith 추적
load_dotenv()

# API키 불러오기
open_api_key = os.environ['OPENAI_API_KEY']
pinecone_api = os.environ['PINECONE_API_KEY']

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  # OpenAI의 임베딩 모델 사용
    api_key=open_api_key)

# 인덱스 가져오기
pc = Pinecone(api_key=pinecone_api)
index_name = "boaz-adv"
index = pc.Index(index_name)

# PineconeVectorStore를 사용하여 벡터 스토어 생성
vectorstore = PineconeVectorStore(index=index, 
                                  embedding=embeddings, 
                                  text_key="page_content")